# Convolutional Neural Network

## Model
![](https://cdn-images-1.medium.com/max/800/0*wigQtmJiv0bddwPI.)

## Source

 - [Medium article](https://towardsdatascience.com/understanding-how-convolutional-neural-network-cnn-perform-text-classification-with-word-d2ee64b9dd0b)
 - [White paper](http://www.aclweb.org/anthology/D14-1181)

## Dados

In [1]:
import keras
import numpy as np
from src.datasetAPI import RotaDosConcursos
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize

C:\Users\pedro\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\pedro\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
dimensions = 50
categories_count = 50

In [3]:
train_data = RotaDosConcursos(subset='train')
train_data.df = train_data.df.iloc[:25]
#test_data = RotaDosConcursos(subset='test')

train_categories_len = len(train_data.target_names)

In [4]:
word_embed_path = 'dataset/glove/glove_s%d.txt'%dimensions
word_embed_model = KeyedVectors.load_word2vec_format(
    word_embed_path,
    unicode_errors="ignore")

In [5]:
sentence_vectors = []
for sentence in train_data.clean_text:
    try:
        sentence_vectors.append([word_embed_model[word] for word in word_tokenize(sentence.lower())])
    except KeyError:
        pass
    except AttributeError:
        print(sentence)

## Normalization

In [6]:
max_sentence_size = max([len(sentence) for sentence in sentence_vectors])

In [7]:
normalized_input = [np.concatenate((sentence,np.zeros((max_sentence_size - len(sentence),dimensions)))) for sentence in sentence_vectors]

## Model

In [8]:
input_layer = keras.Input([max_sentence_size,dimensions])

In [9]:
filter_layer_4_1 = keras.layers.Conv1D(1, 4, activation='relu')(input_layer)
filter_layer_4_2 = keras.layers.Conv1D(1, 4, activation='relu')(input_layer)

filter_layer_3_1 = keras.layers.Conv1D(1, 3, activation='relu')(input_layer)
filter_layer_3_2 = keras.layers.Conv1D(1, 3, activation='relu')(input_layer)

filter_layer_2_1 = keras.layers.Conv1D(1, 2, activation='relu')(input_layer)
filter_layer_2_2 = keras.layers.Conv1D(1, 2, activation='relu')(input_layer)

In [10]:
max_layer_4_1 = keras.layers.MaxPooling1D(max_sentence_size - 4 + 1)(filter_layer_4_1)
max_layer_4_2 = keras.layers.MaxPooling1D(max_sentence_size - 4 + 1)(filter_layer_4_2)

max_layer_3_1 = keras.layers.MaxPooling1D(max_sentence_size - 3 + 1)(filter_layer_3_1)
max_layer_3_2 = keras.layers.MaxPooling1D(max_sentence_size - 3 + 1)(filter_layer_3_2)

max_layer_2_1 = keras.layers.MaxPooling1D(max_sentence_size - 2 + 1)(filter_layer_2_1)
max_layer_2_2 = keras.layers.MaxPooling1D(max_sentence_size - 2 + 1)(filter_layer_2_2)

In [11]:
concat1max = keras.layers.Concatenate(axis=1)([max_layer_4_1,
                                               max_layer_4_2,
                                               max_layer_3_1,
                                               max_layer_3_2,
                                               max_layer_2_1,
                                               max_layer_2_2])

In [12]:
output_layer = keras.layers.Dense(categories_count)(concat1max)

In [13]:
model = keras.Model(input_layer,output_layer)

In [14]:
model.compile(optimizer='adam',loss='categorical_crossentropy')

In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 79, 50)       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 76, 1)        201         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 76, 1)        201         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 77, 1)        151         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (

In [16]:
np.array(normalized_input).shape

(12, 79, 50)

In [17]:
model.evaluate(np.array(normalized_input))

AttributeError: 'NoneType' object has no attribute 'shape'